In [ ]:
import torch
import pandas as pd
import numpy as np
import torch
import os

In [ ]:
def p_acc(target, prediction, pixel_tolerances=[1,3,5,10]):
    """
    Calculate the accuracy of prediction
    :param target: (N, seq_len, 2) tensor, seq_len could be 1
    :param prediction: (N, seq_len, 2) tensor
    :return: a dictionary of p-total correct and batch size of this batch
    """
    # flatten the N and seqlen dimension of target and prediction
    target = target.reshape(-1, 2)
    prediction = prediction.reshape(-1, 2)

    dis = target - prediction
    dist = torch.norm(dis, dim=-1)

    total_correct = {}
    for p_tolerance in pixel_tolerances:
        total_correct[f'p{p_tolerance}'] = torch.sum(dist < p_tolerance)

    bs_times_seqlen = target.shape[0]
    return total_correct, bs_times_seqlen

In [ ]:
def px_euclidean_dist(target, prediction):
    """
    Calculate the total pixel euclidean distance between target and prediction
    in a batch over the sequence length
    :param target: (N, seqlen, 2) tensor
    :param prediction: (N, seqlen, 2) tensor
    :return: total pixel euclidean distance and sample numbers
    """
    # flatten the N and seqlen dimension of target and prediction
    target = target.reshape(-1, 2)[:, :2]
    prediction = prediction.reshape(-1, 2)

    dis = target - prediction
    dist = torch.norm(dis, dim=-1)
    total_px_euclidean_dist = torch.sum(dist)
    sample_numbers = target.shape[0]
    return total_px_euclidean_dist, sample_numbers

In [ ]:
def px_manhaten_dist(target, prediction):
    """
    Calculate the total pixel manhaten distance between target and prediction
    in a batch over the sequence length
    :param target: (N, seqlen, 2) tensor
    :param prediction: (N, seqlen, 2) tensor
    :return: total pixel manhaten distance and sample numbers
    """
    # flatten the N and seqlen dimension of target and prediction
    target = target.reshape(-1, 2)[:, :2]
    prediction = prediction.reshape(-1, 2)
    dis = target - prediction
    dist = torch.sum(torch.abs(dis), dim=-1)
    total_px_manhaten_dist = torch.sum(dist)
    sample_numbers = target.shape[0]
    return total_px_manhaten_dist, sample_numbers

In [ ]:
def calculate_metrics(gt_df, predictions_df):
    # Convert the dataframes into PyTorch tensors
    gt_tensor = torch.tensor(gt_df[['x', 'y']].values, dtype=torch.float32)
    predictions_tensor = torch.tensor(predictions_df[['x', 'y']].values, dtype=torch.float32)

    # Calculate pixel tolerated accuracy
    total_correct, sample_size = p_acc(gt_tensor, predictions_tensor, pixel_tolerances=[1,3,5,10,15])
    # enumerate the tolerances and print the accuracy
    print("p_acc:", end=" ")
    for p_tolerance, correct in total_correct.items():
        print(f'{p_tolerance}={correct.item() / sample_size:.2%}', end=", ")
    print()
    # calculate the px_euclidean_dist
    total_px_euclidean_dist, sample_size = px_euclidean_dist(gt_tensor, predictions_tensor)
    print(f'px_euclidean_dist: {total_px_euclidean_dist.item() / sample_size:.2f}')

    # calculate the px_manhaten_dist
    total_px_manhaten_dist, sample_size = px_manhaten_dist(gt_tensor, predictions_tensor)
    print(f'px_manhaten_dist: {total_px_manhaten_dist.item() / sample_size:.2f}')


In [ ]:
gt_df = pd.read_csv('./gt_orig_merged.csv')

# iterate over csv files in a directory
folder = './your_submissions/'
files = os.listdir(folder)
# remove the files that contains null
files = [file for file in files if 'null' not in file]
# sort files by the Priv_ score
sort_files = sorted(files, key=lambda x: float(x.split('_')[-1].split('.')[0]), reverse=True)
for filename in sort_files:
    if filename.endswith(".csv"):
        print(filename)
        predictions_df = pd.read_csv(os.path.join(folder, filename))

        calculate_metrics(gt_df, predictions_df)
    else:
        continue